# Парсер WB
Получение нужной информации делаем в 2 этапа:
- сначала с помощью bs4 и requests получаем urlы страниц, которые содержать информацию о типах товаров и из кол-ве
-  используя Selenium получаем по каждой ссылке необходимые данные

## Часть 1 Получение ссылок

In [29]:
# Install parsing libraries
# !pip install -r pip-requirements.txt

  Using cached requests-2.24.0-py2.py3-none-any.whl (61 kB)
  Using cached beautifulsoup4-4.9.1-py3-none-any.whl (115 kB)
  Using cached lxml-4.6.1-cp38-cp38-macosx_10_9_x86_64.whl (4.6 MB)
  Using cached soupsieve-2.0.1-py3-none-any.whl (32 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [97]:
from bs4 import BeautifulSoup
import requests
import logging
from time import sleep
import csv
import collections


# Constants
# ROOT_URL = 'https://www.wildberries.ru/catalog/'
ROOT_URL = 'https://www.wildberries.ru'

categories = ['zhenshchinam',
              'muzhchinam',
              'detyam',
              'obuv',
              'aksessuary',
              'elektronika',
              'bytovaya-tehnika',
              'knigi',
              # 'sport',  # сложная категория, другое расположение элементов на странице
              'krasota',
              'igrushki',
              'pitanie',
              'tovary-dlya-zhivotnyh',
              'knigi-i-diski/kantstovary',
              'dom-i-dacha/zdorove',  # категория товаров по здоровью
              'dom-i-dacha/instrumenty',  # категория инструменты
              'dom-i-dacha',  # категория дом
              'aksessuary/avtotovary',  # автотовары
              'podarki',
             ]

In [162]:
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('wb')

# ParseResult = collections.namedtuple(
#     'ParseResult',
#     (
#         'store',
#         'cat_1',
#         'cat_2',
#         'cat_3',
#         'cat_4',
#         'cat_5',
#         'title',
#         'url',
#     ),
# )

# HEADERS = ('store', 'cat_1', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'title', 'url')



class Client:
    
    def __init__(self, categories):
        self.session = requests.Session()
        self.session.headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:82.0) Gecko/20100101 Firefox/82.0",
            "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
            "Referer": "https://www.wildberries.ru/",
            "Connection": "keep-alive",
            "Accept": "*/*",
        }
        self.categories = categories
        self.result = []
        
    def load_page(self, url: str):
        res = self.session.get(url=url)
        res.raise_for_status()
        return res.text
    
    def parse_page(self, text: str, url: str, ):
        # soup = BeautifulSoup(text, 'lxml')  # Видимо этот парсер создает не валидное дерево разбора
        soup = BeautifulSoup(text, 'html.parser')
        
        page_type = self.define_page_type(soup, url)
        if page_type == 1:  # страница с главными категориями
            # Добавляем линки со страницы в список урлов для парсинга
            urls = self.parse_main_categories_urls(soup, url)
            self.parse_pages_from_urls(urls)
        elif page_type == 2:  # страница с подкатегориями, есть <div class="catalog-sidebar">
            urls = self.parse_minor_categories_urls(soup, url)
            self.parse_pages_from_urls(urls)
        elif page_type == 5:  # дошли по крайней категории, добавляем в csv-file данные о товарах в этой подкатегории
            # self.parse_goods_data(soup, url)
            self.write_url_to_txt(url)
            
            
    def define_page_type(self, page, url):
        """
        """
        i_menu_catalog = page.find('div', class_="i-menu-catalog")
        if i_menu_catalog:
            logger.info(f"MAIN i_menu_catalog, url: {url}, return 1")
            return 1  # на странице только категории
        
        # Если есть div с классом catalog-sidebar, то проверяем кол-во
        # дочерних тегов li, если больше 1, то  return 2, иначе return 5
        
        cat_sidebar = page.find("div", class_="catalog-sidebar")
        if not cat_sidebar:
            logger.error(f"no cat_sidebar url: {url}")
            return
        first_li_elem = cat_sidebar.find("li", class_="selected hasnochild")
        ul_in_li = first_li_elem.find("ul")
        if not ul_in_li:
            return 5
        else:
            return 2
             
    def write_url_to_txt(self, url):
        data = f"{url}\n"
        with open('urls_to_parse_data.txt', 'a', encoding='utf-8') as f:
            f.write(data)

    def parse_main_categories_urls(self, soup, url):
        container = soup.find("div", class_="i-menu-catalog")
        if not container:
            logger.error(f"NO container in parse_main_categories_urls, URL: {url}")
            return
        
        ul_elem = container.find("ul", class_="maincatalog-list-2")
        if not ul_elem:
            logger.error(f"NO ul_elem in parse_main_categories_urls, URL: {url}")
            return
        li_elems = ul_elem.find_all("li")
        urls = []
        
        for li_elem in li_elems:
            curr_a_elem = li_elem.find("a")
            urls.append(curr_a_elem['href'])
            
        return urls

    def parse_minor_categories_urls(self, soup, url):
        container = soup.find("div", class_="catalog-sidebar")
        if not container:
            logger.error(f"NO container in parse_minor_categories_urls, url: {url}")
            return
        
        urls = []
        ul_elem = container.find("li", class_="selected hasnochild").find("ul")
        li_elems = ul_elem.find_all("li")
        for li_elem in li_elems:
            urls.append(li_elem.find("a")['href'])
            
        return urls
        
    def parse_pages_from_urls(self, urls: list):
        for url in urls:
            # logger.info(f"url before: {url}")
            url = f"{ROOT_URL}{url}"
            # logger.info(f"url after: {url}")
            soup = self.load_page(url)
            self.parse_page(soup, url)
        
    def prepare_main_urls(self):
        urls = [f"/catalog/{category}" for category in self.categories]
        return urls
    
    def run(self):
        urls = self.prepare_main_urls()
        self.parse_pages_from_urls(urls)

In [163]:
parser = Client(categories)
parser.run()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.wildberries.ru:443
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/zhenshchinam HTTP/1.1" 200 None
INFO:wb:MAIN i_menu_catalog, url: https://www.wildberries.ru/catalog/zhenshchinam, return 1
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/zhenshchinam/odezhda HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/zhenshchinam/odezhda/bluzki-i-rubashki HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/zhenshchinam/odezhda/bryuki-i-shorty HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/zhenshchinam/odezhda/verhnyaya-odezhda HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/zhenshchinam/odezhda/vodolazki HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/zhenshchi

DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/budushchie-mamy/bluzki-koftochki HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/budushchie-mamy/bryuki-dzhinsy HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/budushchie-mamy/verhnyaya-odezhda HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/budushchie-mamy/kostyumy-kombinezony HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/budushchie-mamy/odezhda-dlya-doma HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/budushchie-mamy/odezhda-dlya-kormyashchih-mam HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/budushchie-mamy/pidzhaki-zhilety HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/budushchie-mamy/platya-sara

DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/zhenshchinam/spetsodezhda/golovnye-ubory HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/zhenshchinam/spetsodezhda/rabochaya-obuv HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/zhenshchinam/spetsodezhda/odnorazovye-izdeliya HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/zhenshchinam/spetsodezhda/sredstva-zashchity-organov-dyhaniya HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/zhenshchinam/spetsodezhda/sredstva-individualnoy-zashchity HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/muzhchinam HTTP/1.1" 200 None
INFO:wb:MAIN i_menu_catalog, url: https://www.wildberries.ru/catalog/muzhchinam, return 1
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/muzhchinam/odezhd

DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/detyam/odezhda/dlya-devochek/vodolazki HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/detyam/odezhda/dlya-devochek/dzhempery-i-kardigany HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/detyam/odezhda/dlya-devochek/dzhinsy-dzhegginsy HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/detyam/odezhda/dlya-devochek/zhilety HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/detyam/odezhda/dlya-devochek/karnavalnye-kostyumy HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/detyam/odezhda/dlya-devochek/kombinezony HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/detyam/odezhda/dlya-devochek/kostyumy HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wild

DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/detyam/dlya-novorozhdennyh/pinetki HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/detyam/odezhda/dlya-novorozhdennyh/platya-yubki HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/detyam/odezhda/dlya-novorozhdennyh/raspashonki HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/detyam/odezhda/dlya-novorozhdennyh/futbolki-topy HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/elektronika/detskaya-elektronika HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/detyam/konstruktory HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/sport/dlya-detey/detskiy-transport HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/detyam/d

...

DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/dlya-doma/hozyaystvennye-tovary/vannaya HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/dlya-doma/hozyaystvennye-tovary/vannaya/osvezhiteli-i-aksessuary HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/dom/hozyaystvennye-tovary/vannaya-i-tualet/pokrytiya-dlya-unitaza HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/dlya-doma/hozyaystvennye-tovary/vannaya/chistyashchie-sredstva HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/dlya-doma/bytovaya-himiya/ochistiteli-dlya-unitaza HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/dlya-doma/hozyaystvennye-tovary/vannaya/chistyashchie-sredstva/sredstva-dlya-septikov HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catal

DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/muzhchinam HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/muzhchinam/drugu HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/muzhchinam/pape HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/muzhchinam/dedushke HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/muzhchinam/kollege HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/muzhchinam/lyubimomu HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki-/muzhchinam/lyubimomu/avtotovary HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/muzhchinam/lyubimomu/avtokruzhki HTTP/1.1" 200 None
DEBUG:urllib3.connecti

DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/muzhchinam/lyubimomu/podstakanniki HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/muzhchinam/lyubimomu/dekorativnye-podushki HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/muzhchinam/lyubimomu/polotentsa-bannye HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/muzhchinam/lyubimomu/ryumki HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki-/muzhchinam/lyubimomu/dlya-doma/skovorody HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/muzhchinam/lyubimomu/stakany HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/muzhchinam/lyubimomu/termokruzhki HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wi

DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/na-sobytie/8-marta/chasy-naruchnye HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/na-sobytie/8-marta/chehly-dlya-telefonov HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/na-sobytie/23-fevralya HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki-/na-sobytie/vypusknoy HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki-/na-sobytie/vypusknoy/detskiy-sad HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki-/na-sobytie/vypusknoy/shkola HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki/na-sobytie/den-rozhdeniya HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://www.wildberries.ru:443 "GET /catalog/podarki-/

AttributeError: 'Client' object has no attribute 'write_to_csv'

## Часть 2 Получение  данных по ссылкам

В этой части программы стоит задача пройтись по каждой ссылке, полученной парсером Client и получить данные с каждой страницы с дальнейшим сохранением в csv-файл.

In [202]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from selenium.common.exceptions import TimeoutException

from selenium.common.exceptions import NoSuchElementException
from pygeckodriver import geckodriver_path
from time import sleep
from datetime import datetime

In [218]:
# logging.basicConfig(level=logging.DEBUG)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('WB part_2')

In [221]:
class DataParser:
    
    def __init__(self, read_path, csv_path):
        self.read_path = read_path  # путь к txt файлу с урлами
        self.csv_path = csv_path # путь к txt файлу с урлами
        self.keys = ['store', 'categories', 'goods_quantity', 'title',
                     'url']  # ключи, короткие названия данных для csv-файла
        
    def write_csv_headers(self, path, keys):
        with open(path, 'w') as f:
            w = csv.DictWriter(f, keys)
            w.writeheader()
    
    def write_to_csv(self, data_list: list):
        with open(self.csv_path, 'a') as f:
            for data in data_list:
                w = csv.DictWriter(f, self.keys)
                w.writerow(data)
      
    def get_common_page_info(self, driver, url):
        """
        Получает со страницы общую информацию:
        - Категории, начиная со следующей после Главной категории
        - Всего товаров в этой категории
        Например, категории товара: Главная/Детям/Для мальчико/Белье
        Получим все категории, кроме Главная
        
        return a dict-object
        """
        i_bread_crumbs_x_path = "/html/body/div[2]/div/div/div[1]"
        i_bread_crumbs = driver.find_element_by_xpath(i_bread_crumbs_x_path)
        cat_elems = i_bread_crumbs.find_elements_by_class_name("secondary")
        categories = []
        
        for elem in cat_elems:
            cat = elem.find_element_by_tag_name("span").text
            categories.append(cat)
        # make string from list of categories
        categories = '|'.join(categories)
        goods_quantity = driver.find_element_by_class_name("goods-count.j-goods-count").text.split(' ')[0]
        
        # Инициируем словарь и запишем в него данных по ключам
        data = dict.fromkeys(self.keys, None)
        data['store'] = 'WB'
        data['categories'] = categories
        data['goods_quantity'] = goods_quantity
        data['url'] = url
        return data

    def get_subjects_data(self, driver, url):
        """
        Собирает данные со страницы, если имеется несколько подтипов товаров
        в этой категории, например:
        Категория: Белье для мальчиков
        Подтипы товаров:
        - Боди
        - Гетры
        - Кальсоны
        и т.д.
        """
        page_data_list = []
        common_data = self.get_common_page_info(driver, url)
        subjects_block = driver.find_element_by_class_name("j-list.list_left.list_left_xsubject.render_type_1")
        subjects = subjects_block.find_elements_by_tag_name("label")
        
        for item in subjects:
            item_data = common_data.copy()
            item_data['title'] = item.text
            page_data_list.append(item_data)
        
        logger.info(f"Collected {len(subjects)} items, url: {url}")
        return page_data_list
            
    def get_subject_data(self, driver, url):
        """
        Собирает данные со страницы, если нет подтипов товара
        в этой категории.
        """
        data = self.get_common_page_info(driver, url)
        
        # ищем значение последней категории
        i_bread_crumbs_x_path = "/html/body/div[2]/div/div/div[1]"
        i_bread_crumbs = driver.find_element_by_xpath(i_bread_crumbs_x_path)
        last_cat_elem = i_bread_crumbs.find_elements_by_class_name("secondary")[-1]

        data['title'] = last_cat_elem.text
        logger.info(f"Collected {len([data])} items, url: {url}")
        return [data]
    
    def check_exists_by_id(self, driver):
        """
        Проверяет наличие тега, присутсвующего только на страницах без товаров.
        """
        try:
            zero_content_elem = driver.find_element_by_id("divGoodsNotFound")
        except NoSuchElementException:
            return False
        
        return True

    def define_page_type(self, driver, url):
        """
        Определяет тип страницы: с подтипами товаров
        или нет
        return int
        """
        is_goods_not_exist = self.check_exists_by_id(driver)  # Проверяет наличие тега при пустой странице товара
        if is_goods_not_exist:
            logger.info(f"Товаров на странице НЕТ, url: {url}")
            return 0
        
        wait = WebDriverWait(driver, 20)
        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "filter-title-text")))

        try:
            subjects_block = driver.find_element_by_class_name("filterblock.render_type_1.xsubject.show.filter-active")
            logger.info(f"ЕСТЬ подтипы товара, url: {url}")
            return 1
        except NoSuchElementException:
            logger.info(f"НЕТ подтипов товара, url: {url}")
            return 2
        logger.error(f"Faild define_page_type, url: {url}")
        
        
    def parse_data_from_page(self, driver, url):
        wait = WebDriverWait(driver, 20)
        driver.get(url)
        # ждем пока не подгрузится тег h1 - название категории
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "c-h1")))
        
        data_list = []
        page_type = self.define_page_type(driver, url)
        if page_type == 0:
            return
        elif page_type == 1:  # на странице есть подтипы товаров
            data_list.extend(self.get_subjects_data(driver, url))
        elif page_type == 2:  # на странице нет подтипов товара
            data_list.extend(self.get_subject_data(driver, url))
        
        logger.info(f"Collected: {len(data_list)} items, url: {url}")
        self.write_to_csv(data_list)
        
    def url_generator(self, read_path):
        with open(read_path) as f:
            for line in f.readlines():
                yield line.strip()
                
    def run(self):
        driver = webdriver.Firefox(executable_path=geckodriver_path)
        self.write_csv_headers(self.csv_path, self.keys)  # Запишем заголовки в csv-файл
        for url in self.url_generator(self.read_path):
            logger.info(f"Current url: {url}")
            self.parse_data_from_page(driver, url)
        logger.info(f"Parsing complete")

In [222]:
data_parser = DataParser("urls_to_parse_data.txt", "goods_data.csv")
start = datetime.now()
data_parser.run()
end = datetime.now()
print(f"Total duration: {(end - start)/60} mins.")

DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "firefox", "acceptInsecureCerts": true}}, "desiredCapabilities": {"browserName": "firefox", "acceptInsecureCerts": true, "marionette": true}}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:63340
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session HTTP/1.1" 200 735
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
INFO:WB part_2:Current url: https://www.wildberries.ru/catalog/zhenshchinam/odezhda/bluzki-i-rubashki
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/url {"url": "https://www.wildberries.ru/catalog/zhenshchinam/odezhda/bluzki-i-rubashki"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/url HTTP/1.1" 200 14
DEBUG:selenium.webdriver.remote.r

DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/execute/sync HTTP/1.1" 200 14
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "css selector", "value": ".filterblock.render_type_1.xsubject.show.filter-active"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element HTTP/1.1" 200 88
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
INFO:WB part_2:ЕСТЬ подтипы товара, url: https://www.wildberries.ru/catalog/zhenshchinam/odezhda/bluzki-i-rubashki
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "xpath", "value": "/html/body/div[2]/div/div/div[1]"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da

DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "css selector", "value": ".filter-title-text"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element HTTP/1.1" 404 296
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "css selector", "value": ".filter-title-text"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element HTTP/1.1" 200 88
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/execute/sync {"script": "return (function(){return function(){var k=this;function l(a){return void 0!==a}fun

DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/execute/sync HTTP/1.1" 200 14
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "css selector", "value": ".filterblock.render_type_1.xsubject.show.filter-active"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element HTTP/1.1" 200 88
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
INFO:WB part_2:ЕСТЬ подтипы товара, url: https://www.wildberries.ru/catalog/zhenshchinam/odezhda/bryuki-i-shorty
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "xpath", "value": "/html/body/div[2]/div/div/div[1]"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c

DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "GET /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element/9dcdd103-cdb3-c94d-a43e-c6d2749e1e77/text HTTP/1.1" 200 35
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:selenium.webdriver.remote.remote_connection:GET http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element/28e50962-3489-4541-94af-de72f47868f5/text {"id": "28e50962-3489-4541-94af-de72f47868f5"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "GET /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element/28e50962-3489-4541-94af-de72f47868f5/text HTTP/1.1" 200 31
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
INFO:WB part_2:Collected 8 items, url: https://www.wildberries.ru/catalog/zhenshchinam/odezhda/bryuki-i-shorty
INFO:WB part_2:Collected: 8 items, url: https://www.wildberries.ru/catalog/zhenshchinam/odezhda/bryuki-i-shorty
INFO:WB part_2:Current url: https://www.wildberries.ru/catalog/zhenshchinam/odezhda/

...

DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/execute/sync HTTP/1.1" 200 14
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "css selector", "value": ".filterblock.render_type_1.xsubject.show.filter-active"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element HTTP/1.1" 200 88
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
INFO:WB part_2:ЕСТЬ подтипы товара, url: https://www.wildberries.ru/catalog/zhenshchinam/odezhda/futbolki-i-topy
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "xpath", "value": "/html/body/div[2]/div/div/div[1]"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c

DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "css selector", "value": "[id=\"divGoodsNotFound\"]"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element HTTP/1.1" 404 303
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "css selector", "value": ".filter-title-text"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element HTTP/1.1" 404 296
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "css selector", "value": ".filter-title-text"}
DEBUG:urllib3.connectionpool:http:/

DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/execute/sync HTTP/1.1" 200 14
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "css selector", "value": ".filterblock.render_type_1.xsubject.show.filter-active"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element HTTP/1.1" 404 332
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
INFO:WB part_2:НЕТ подтипов товара, url: https://www.wildberries.ru/catalog/zhenshchinam/odezhda/hudi
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "xpath", "value": "/html/body/div[2]/div/div/div[1]"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-

DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/execute/sync HTTP/1.1" 200 14
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "css selector", "value": ".filterblock.render_type_1.xsubject.show.filter-active"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element HTTP/1.1" 404 332
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
INFO:WB part_2:НЕТ подтипов товара, url: https://www.wildberries.ru/catalog/zhenshchinam/odezhda/yubki
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:63340/session/da4c9fb2-3451-3344-ad50-a44257ec3f40/element {"using": "xpath", "value": "/html/body/div[2]/div/div/div[1]"}
DEBUG:urllib3.connectionpool:http://127.0.0.1:63340 "POST /session/da4c9fb2-3451

KeyboardInterrupt: 

# Архив кода 

In [164]:
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.common.exceptions import TimeoutException
# from selenium.common.exceptions import NoSuchElementException
# from pygeckodriver import geckodriver_path
# from time import sleep

# ROOT_URL = 'https://www.wildberries.ru/'
# CATALOG = 'catalog'

# categories = ['zhenshchinam',
#               'muzhchinam',
#               'detyam',
#               'obuv',
#               'aksessuary',
#               'elektronika',
#               'bytovaya-tehnika',
#               'knigi',
#               # 'sport',  # сложная категория, другое расположение элементов на странице
#               'krasota',
#               'igrushki',
#               'pitanie',
#               'tovary-dlya-zhivotnyh',
#               'kantstovary',
#               'dom-i-dacha/zdorove',  # категория товаров по здоровью
#               'dom-i-dacha/instrumenty',  # категория инструменты
#               'dom-i-dacha',  # категория дом
#               'aksessuary/avtotovary',  # автотовары
#               'podarki',
#              ]

In [166]:
# # Список для всех url-ов категорий
# all_categories_urls = []

In [9]:
def get_top_menus(driver, unnecessary_cat_ids=[]):
    """
    Функция получается ссылки на основные
    категории товаров Wildberries. В итоговый список
    не включены те категории, id которых переданы в
    списке.
    Возвращает список url-ов.
    
    """
    driver.get(ROOT_URL)
    sleep(1)

    main_menu_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/div/div[2]/button')
    main_menu_button.send_keys(Keys.RETURN)

    # Поиск всех элементов topmenus
    top_menus_block = driver.find_element_by_class_name("topmenus")
    top_menus_items = top_menus_block.find_elements_by_class_name("topmenus-item")
    print(f"Len top menus: {len(top_menus_items)}")

    # Получим ссылки на top_menus и сразу исключим не нужные категории
    menus_links = []
    for item in top_menus_items:
        # Получим значение data-menu-id
        cur_id = int(item.get_attribute('data-menu-id'))
        if cur_id not in unnecessary_cat_ids:
            a_elem = item.find_element_by_tag_name('a')
            cur_url = a_elem.get_attribute('href')
            menus_links.append(cur_url)

    return menus_links

In [10]:
def parse_categories_on_page(driver):
    """
    Парсит все ссылки со страницы в боковом 
    меню.
    """
    categories_licks_on_page = []
    # проверка на sport
    last_url_part = driver.current_url.split('/')[-1]
    if last_url_part == 'sport':
        ul_elem = driver.find_element_by_class_name("j-menu-catalog-second")

        # сразу ищет теги a по классу, удалям сразу первую
        a_elems = ul_elem.find_elements_by_class_name("menu-item-link")[1:]
    else:
        try:
            ul_elem = driver.find_element_by_class_name("sidemenu")
            a_elems = ul_elem.find_elements_by_tag_name('a')[1:]
        except NoSuchElementException:
            try:
                ul_elem = driver.find_element_by_class_name("maincatalog-list-2")
                a_elems = ul_elem.find_elements_by_tag_name("a")
            except NoSuchElementException:
                print(f"Parser failed on {driver.current_url}")
        
        
    #  добавление href в categories_licks_on_page
    for a_elem in a_elems:
        cur_url = a_elem.get_attribute('href')
        categories_licks_on_page.append(cur_url)

    return categories_licks_on_page

In [11]:
def collect_urls_from_list(driver, urls):
    """

    """
    new_categories = []

    for url in urls:
        driver.get(url)
        new_categories += parse_categories_on_page(driver)

    return new_categories

In [12]:
def main():

    driver = webdriver.Firefox(executable_path=geckodriver_path)
    # исключенные категории:
    # # Авиабилеты id="61037", Premium id="63451", Ювелирные изделия id="1023",
    # Тренды id="1003", Бренды id="4853", Акции id="2192", Цифровые товары id="62813"
    unnecessary_cat_ids = [61037, 63451, 1023, 1003, 4853, 2192, 62813]
    top_menus_urls = get_top_menus(driver, unnecessary_cat_ids)

    second_menus_urls = collect_urls_from_list(driver, top_menus_urls)

    print(f"len second_menus_urls: {len(second_menus_urls)}")
    for url in second_menus_urls:  
        print('-->', url)

    # exit
    print(f"End programm")
    driver.close()

In [13]:
if __name__ == '__main__':
    main()

Len top menus: 27
len second_menus_urls: 272
--> https://www.wildberries.ru/catalog/zhenshchinam/odezhda
--> https://www.wildberries.ru/catalog/zhenshchinam/odezhda/bluzki-i-rubashki
--> https://www.wildberries.ru/catalog/zhenshchinam/odezhda/bryuki-i-shorty
--> https://www.wildberries.ru/catalog/zhenshchinam/odezhda/verhnyaya-odezhda
--> https://www.wildberries.ru/catalog/zhenshchinam/odezhda/vodolazki
--> https://www.wildberries.ru/catalog/zhenshchinam/odezhda/dzhempery-i-kardigany
--> https://www.wildberries.ru/catalog/zhenshchinam/odezhda/dzhinsy-dzhegginsy
--> https://www.wildberries.ru/catalog/zhenshchinam/odezhda/zhilety
--> https://www.wildberries.ru/catalog/zhenshchinam/odezhda/karnavalnye-kostyumy
--> https://www.wildberries.ru/catalog/zhenshchinam/odezhda/kigurumi
--> https://www.wildberries.ru/catalog/zhenshchinam/odezhda/kombinezony-polukombinezony
--> https://www.wildberries.ru/catalog/zhenshchinam/odezhda/kostyumy
--> https://www.wildberries.ru/catalog/zhenshchinam/odezh